# 크롤링 실습

## 과기정통부 홈페이지(msit.go.kr)의 보도자료 hwp파일 수집

<img src="https://i.ibb.co/zZmw0YX/254.png" width=1550>

# 모듈 임포트

In [1]:
from time import sleep  # 몇 초간 멈추고 기다려줘야 할 때

from selenium import webdriver  # 크롬브라우저를 원격으로 컨트롤하기 위한 모듈
import chromedriver_autoinstaller  # 크롬브라우저 드라이버(원격 컨트롤을 위한 프로그램) 간편다운로드 모듈
from selenium.common.exceptions import TimeoutException  # 브라우저에서 발생하는 특정 오류 핸들링을 위한 클래스
from selenium.webdriver.common.by import By  # 어떤 속성으로 요소를 찾을지(css_selector, id..) 결정할 때 쓰는 간편한 클래스
from selenium.webdriver.support import expected_conditions as EC  # 브라우저 특정 상태에서의 반응을 결정할 수 있는 클래스
from selenium.webdriver.support.wait import WebDriverWait  # time.sleep 말고 EC에 따른 대기조건을 결정할 수 있는 클래스

from selenium.webdriver.chrome.options import Options  # 옵션 설정(헤드리스, 묻지마 다운로드, user-agent 위변조 등..)

# 크롬옵션 설정(묻지 않고 다운로드 시작)

# & 크롬드라이버 자동다운로드

In [2]:
chrome_options = Options()
prefs = {'profile.default_content_setting_values.automatic_downloads': 1}
chrome_options.add_experimental_option("prefs", prefs)
driver_path = chromedriver_autoinstaller.install()

# 브라우저(크롬드라이버) 열기

In [3]:
driver = webdriver.Chrome(driver_path, options=chrome_options)

# 과기정통부 보도자료 페이지 접속

In [4]:
driver.get("https://www.msit.go.kr/bbs/list.do?sCode=user&mPid=112&mId=113")

# (현재 1페이지이지만 확인차) 다시 1페이지로 이동

In [5]:
sleep(3)
driver.execute_script("fn_paging(1);")

# 첫 번째(최상단) 글 클릭

In [6]:
driver.find_element_by_css_selector('#td_NTT_SJ_0').click()

# 본격적인 다운로드 반복작업 시작

In [ ]:
while True:
    try:  # 오류 핸들링을 위한 try-except 구문

        # 먼저 "다운로드"를 위한 <a> 태그 찾기
        download_element = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR,
                           """#search-view-form > div > div.view_file > ul > li > span > a:nth-child(1)""")))

        # <a href="javascript:void(0);" onclick="fn_download('38668', '1', 'hwpx');" class="down" title="파일다운로드">다운로드</a>                                 ===================================

        # <a> 태그의 onclick 속성값 추출 : "fn_download('38668', '1', 'hwpx');"
        download_script = download_element.get_attribute('onclick')

        # onclick 속성(JS코드) 실행 -> 다운로드 시작
        driver.execute_script(download_script)

    except TimeoutException as e:  # 첨부파일이 없는 경우(페이지 오류) 프로그램이 종료되지 않게 처리함
        print(e)  # 오류메시지 출력
        print(driver.current_url)  # 첨부파일 없는 페이지 주소 출력

    try:
        # 다음글로 이동하는 onclick attribute(JS코드) 추출
        pre_link_element = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR,
                                              '#search-view-form > div > div.page_skip > ul > li:nth-child(2) > p > a')))
        pre_link_script = pre_link_element.get_attribute('onclick')

        # 다음글로 이동하는 JS코드 실행
        driver.execute_script(pre_link_script)

    except TimeoutException as e:  # 가끔 렉이 심하게 걸리거나, 페이지가 없는 경우 오류 핸들링하기 위함(종료 방지)
        print(e)  # 오류메시지 출력하고
        driver.back()  # 다시 이전 페이지(이전글)로 이동
        sleep(3)  # 3초 쉬고 재시도
        pre_link_element = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#search-view-form > div > div.page_skip > ul > li:nth-child(2) > p > a')))
        pre_link_script = pre_link_element.get_attribute('onclick')
        #%% # 이전글로 이동 코드 실행
        driver.execute_script(pre_link_script)

Message: 

https://www.msit.go.kr/bbs/view.do?sCode=user&mId=113&mPid=112&bbsSeqNo=94&nttSeqNo=3181970&formMode=&pageIndex=1&searchCtgry=&searchCtgry2=&searchCtgry3=&RLS_YN=&searchOpt=ALL&searchTxt=


완료 코드는 따로 짜 놓지 않았음

"이전글" 버튼 없으면 알아서 오류 뱉고 종료되겠지..